# Building an AI Sports Video Analysis Agent

## Business Objective

This notebook guides you through building an AI agent specialized in sports video analysis using Amazon Bedrock. Media broadcasters receive hundreds of thousands of video clips from sporting events that each need to be manually analyzed and tagged for downstream distribution channels. Some clips are published to websites, others to social media, and some are archived.

### The Challenge

Each distribution channel has strict content compliance requirements. For example:

**Content Requirements (Sports):**
- Include complete event/game name designation in all communications
- Provide accurate venue details and location information
- Clearly state the participating team names
- Reference key player names prominent in the video

**Distribution Requirements (Social Media):**
- Limit content descriptions to 75 words maximum
- Incorporate 2-3 relevant hashtags per post
- Tag all applicable team/player social media handles
- Maintain an approachable tone with light emoji

The problem: Much of this required information is not present in the video footage itself. These gaps have traditionally required countless hours of manual labor, with human operators manually researching and filling in missing information.

### The Solution

Our solution employs autonomous agents to automatically identify these gaps, perform lookups across various systems, and transform the data into compliant output formats.

### Main Steps

1. **Video Analysis:** Extract metadata from sports video clips using Amazon Bedrock Data Automation (BDA) to understand what's happening in the footage.

2. **Knowledge Base Creation:** Build a sports match information knowledge base containing game details, venues, dates, and team information that can be searched semantically.

3. **Player Database Integration:** Set up a DynamoDB table with player information (names, numbers, positions) that can be queried to identify players seen in videos.

4. **Sports Agent Development:** Create an AI agent that combines video analysis with knowledge base search and player lookups to automatically generate compliant content metadata.

<div class="alert-warning">
    <b>Warning:</b>
    <p>
    1) please make sure to run <b>00-prerequisites.ipynb</b> to properly setup all the packages.
    </p>
</div>

## Initialize the parameters

In [ ]:
import boto3
import json
import uuid
import sagemaker
import sys
from pathlib import Path
from IPython.display import Video, display, Markdown, JSON

# Import display helpers
sys.path.insert(0, str(Path.cwd().parent / 'helper'))
from display_helper import print_success, print_info, print_result

boto_session = boto3.session.Session()
sess = sagemaker.Session(boto_session=boto_session)
if sess.default_bucket():
    bucket = sess.default_bucket()
else:
    bucket = "<YOUR-BUCKET-NAME>" # Provide your own bucket
region = sess.boto_region_name
prefix = "sports-agent"

# set up S3 client
s3_client = boto_session.client('s3')

# access account id
sts_client = boto_session.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

## Analyze Sports Video

The video you'll analyze is a basketball game clip. The video analysis metadata is generated by [Bedrock Data Automation (BDA)](https://docs.aws.amazon.com/bedrock/latest/userguide/bda.html), a managed video understanding feature in Bedrock that extracts key information from video content including scene descriptions, actions, and visual elements.

In [ ]:
video_file = 'sports_01.mp4'

url = f"https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/7db2455e-0fa6-4f6d-9973-84daccd6421f/{video_file}"

!curl {url} -o {video_file}

### Upload Video to S3

In [ ]:
s3_key = f"{prefix}/{video_file}"

s3_client.upload_file(video_file, bucket, s3_key)
video_s3_path = f"s3://{bucket}/{s3_key}"

### Analyze the Video

The video analysis metadata is generated by [Bedrock Data Automation (BDA)](https://docs.aws.amazon.com/bedrock/latest/userguide/bda.html), a managed video understanding feature in Bedrock that extracts scene descriptions, actions, and visual elements from video content.

This process takes 5-10 minutes to complete.

In [ ]:
import sys
sys.path.insert(0, '..')

from helper.bda_helper import BDAVideoProcessor

# Process video
print_info(f"Processing video: {video_file}")
print_info("This will take 5-10 minutes...")

processor = BDAVideoProcessor(region=region, bucket=bucket)
result = processor.process_video(
    video_path=video_file,
    wait=True,
    verbose=True
)

bda_result = result['results']
print_success("Video processing complete!")


In [ ]:
# Preview video and the metadata
display(Video(video_file, width=800))

print("### Video Analysis Metadata:")

display(Markdown(bda_result["video"]["summary"]))

## Setup Sports Agent

Now that we have our video analysis results, let's create an AI agent using Strands Agents framework. This agent will be specialized in sports video analysis and can:
- Process video content analysis from BDA
- Search through sports match databases
- Look up player information
- Combine multiple sources to generate compliant content metadata

### Why Use an Agent?
Agents excel at orchestrating complex tasks that require multiple steps and different types of analysis. For sports content compliance, our agent will:
1. Analyze the video content we extracted with BDA
2. Query a knowledge base of match information (venues, dates, teams)
3. Look up player details from a database
4. Combine these insights to fill in missing information gaps

This automated approach helps media companies process content more efficiently than manual review while ensuring compliance with distribution requirements.

Let's start by setting up the necessary AWS clients and configurations:

### Import Helper Functions

We'll import helper classes that simplify working with AWS services:
- `DynamoDBHelper`: Manages DynamoDB table operations (create, load, query, delete)
- `KnowledgeBasesForAmazonBedrock`: Manages Knowledge Base operations (create, sync, search)

In [ ]:
import sys
sys.path.insert(0, '..')

from helper.dynamodb_helper import DynamoDBHelper

from helper.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
dynamo = DynamoDBHelper()
kb = KnowledgeBasesForAmazonBedrock()

## Load Shared Resources

Load the Knowledge Base and DynamoDB table created in the prerequisites notebook. These resources are shared across all labs to save time and cost.

**What We're Loading:**
- **Sports Knowledge Base** - Contains match information (teams, venues, dates)
- **Players DynamoDB Table** - Contains player details (names, numbers, positions)

These resources were created by running `../00-prerequisites/00-prerequisites.ipynb`.

In [ ]:
# Restore shared resources from prerequisites
%store -r sports_kb_id
%store -r players_table

# Use the shared resources
lab_kb_id = sports_kb_id
lab_dynamo_table = players_table

print_success(f"Loaded Sports Knowledge Base: {lab_kb_id}")
print_success(f"Loaded Players Table: {lab_dynamo_table}")

### Verify Knowledge Base

Test the Knowledge Base by performing a sample search to ensure it's working correctly.

In [ ]:
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')

test_response = bedrock_agent_runtime.retrieve(
    knowledgeBaseId=lab_kb_id,
    retrievalQuery={'text': bda_result["video"]["summary"]},
    retrievalConfiguration={
        'vectorSearchConfiguration': {
            'numberOfResults': 1
        }
    }
)

if test_response.get('retrievalResults'):
    print_success(f"✓ Knowledge Base is working! Found {len(test_response['retrievalResults'])} results")
    # Show first result
    result = test_response['retrievalResults'][0]
    print_info(f"Sample result: {result['content']['text'][:100]}...")
else:
    print_info("Knowledge Base is ready but no results found")

### Verify DynamoDB Table

Test the DynamoDB table by querying for sample player data.

In [ ]:
# Test DynamoDB table with a sample query
test_query = dynamo.query_table(lab_dynamo_table, 'team_name', 'Alpine Wolves')

if test_query:
    print_success("DynamoDB table is working!")
    print_info(f"Found {len(test_query)} players for Alpine Wolves")
    print_info(f"Sample player: {test_query[0].get('player_name', 'N/A')}")
else:
    print_info("DynamoDB table is ready but no results found")

## Create Sports Video Analysis Agent

Now we'll create the agent using the Strands framework with access to the shared resources.

### Define Agent Tools

Defining tools in Strands is simple - just add a `@tool` decorator to your function and provide a description in the docstring. The agent will use this information to understand when and how to use each tool.

#### Tool 1: Retrieve Match Information

**Purpose:** This tool helps retrieve relevant match information from the sports knowledge base using semantic search. It provides context about teams, venues, dates, and game details that can enrich the video analysis.

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel

# Initialize AWS clients
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')
dynamodb = boto3.resource('dynamodb')

agent_model_id = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0'

Initialize AWS clients for the agent tools.

In [ ]:
@tool
def retrieve_match_info(query: str, max_results: int = 1) -> str:
    """
    Search the sports knowledge base for match reports containing team names, venues, dates, scores, and game details.
    Use this to identify which teams are playing in the video.
    
    Args:
        query: Search query describing the game or video content
        max_results: Maximum number of results to return (default: 1)
        
    Returns:
        JSON string with match information including team names, venue, date, and game summary
    """
    try:
        
        response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=lab_kb_id,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results
                }
            }
        )
        return json.dumps(response.get('retrievalResults', []))
    except Exception as e:
        return json.dumps({"error": str(e), "note": "Falling back to video analysis only"})

print_success("retrieve_match_info tool ready")

#### Tool 2: Lookup Player Information

Queries the shared DynamoDB table for player details.

In [ ]:
@tool
def lookup_player_info(team_name: str, player_number: str) -> str:
    """
    Look up detailed player information (name, position, height, weight) from the player database.
    Requires the exact team name from match reports and the player's jersey number from the video.
    
    Args:
        team_name: Exact team name with proper capitalization (e.g., "Alpine Wolves", "Metropolitan Knights")
        player_number: Player's jersey number (e.g., "14", "8", "17")
        
    Returns:
        JSON string with player details including name, position, height, and weight
    """
    try:
        table = dynamodb.Table(lab_dynamo_table)
        
        response = table.get_item(
            Key={
                'team_name': team_name,
                'player_number': str(player_number)
            }
        )
        
        if 'Item' in response:
            return json.dumps(response['Item'])
        else:
            return json.dumps({
                "team_name": team_name,
                "player_number": player_number,
                "player_name": f"Player #{player_number}",
                "position": "Unknown",
                "status": "not_found"
            })
            
    except Exception as e:
        return json.dumps({
            "error": str(e),
            "team_name": team_name,
            "player_number": player_number,
            "status": "lookup_failed"
        })

print_success("lookup_player_info tool ready")

### Initialize the Agent

Create the Sports Video Analysis Agent with:
- Foundation model (Claude 3.7 Sonnet)
- Two custom tools
- System prompt defining the agent's role and behavior

In [ ]:
SYSTEM_PROMPT = """You are a sports video analysis assistant that answers user queries about the provided sports video.

You have video metadata, as well as additional tools to get more information from match reports and player table.

Base on the information you obtained from metadata and tools, generate a clear and accurate answer to the user query. DO NOT answer queries beyond the game you are reviewing.
"""

model = BedrockModel(
    model_id=agent_model_id,
    temperature=0.3,
    region_name=region,
)

# Create the sports video analysis agent with all tools
agent = Agent(
    model=model,
    tools=[
        retrieve_match_info,  # Tool 1: Knowledge Base search
        lookup_player_info,   # Tool 2: DynamoDB player lookup
    ],
    system_prompt=SYSTEM_PROMPT,
)

print_success("Sports Video Analysis Agent created successfully!")

## Test the Agent

Test the agent by asking it to identify players in the video. The agent will:
1. Analyze the BDA video summary
2. Search the knowledge base for match information
3. Query DynamoDB for player details
4. Combine all sources to answer the query

Example queries:
- Who are the players in the video?
- When did this game take place?
- What is the final score of this game?

In [ ]:
query = "who are the players in the video?"

In [ ]:
PROMPT_TEMPLATE="""
VIDEO SUMMARIES: {video_summaries}

USER QUERY: {query}
"""

prompt = PROMPT_TEMPLATE.replace("{video_summaries}", bda_result["video"]["summary"])
prompt = prompt.replace("{query}", query)


In [ ]:
result = agent(prompt=prompt)

# Parse result
try:
    response_text = result.get('output', result) if isinstance(result, dict) else str(result)
    print_result("Agent Response", response_text)
except Exception as e:
    print(f"Error parsing response: {e}")
    print(f"Raw response: {str(result)}")

### Store Parameters for Next Lab

Save the Knowledge Base ID, DynamoDB table name, and BDA results for use in subsequent labs.

In [ ]:
%store lab_kb_id
%store lab_dynamo_table
%store bda_result

## 🎉 Lab Complete!

You've successfully created a Sports Video Analysis Agent that automates content compliance! Here's what you accomplished:

**What You Built:**
- Processed sports video with Amazon Bedrock Data Automation (BDA)
- Created a Knowledge Base with sports match information
- Set up a DynamoDB table with player data
- Built an agent with 2 custom tools that automatically fill information gaps

**The Impact:**
Instead of manual research to find venue details, player names, and game information, the agent automatically:
- Identifies what's in the video (from BDA)
- Looks up missing context (from Knowledge Base)
- Retrieves player details (from DynamoDB)
- Combines everything into compliant metadata

This solution transforms hours of manual work into seconds of automated processing, helping media broadcasters meet strict content compliance requirements for different distribution channels.

### Resources

- [Amazon Bedrock Data Automation](https://docs.aws.amazon.com/bedrock/latest/userguide/data-automation.html)
- [Strands Agents Documentation](https://strandsagents.com/latest/)
- [Amazon Bedrock Knowledge Bases](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)